In [1]:
import numpy as np
import pandas as pd
import json
import ast
import re

In [2]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playergamelog, teamgamelog
from nba_api.stats.endpoints import BoxScoreDefensive
from nba_api.stats.library import data
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import boxscorematchups

pd.set_option('display.max_columns', None)

In [3]:
player_dict = players.get_players()

In [5]:
steph = [player for player in player_dict if player['full_name'] == 'Stephen Curry'][0]

In [6]:
steph

{'id': 201939,
 'full_name': 'Stephen Curry',
 'first_name': 'Stephen',
 'last_name': 'Curry',
 'is_active': True}

In [7]:
steph_id = steph['id']

In [8]:
team_dict = teams.get_teams()

In [10]:
gsw = [team for team in team_dict if team['abbreviation'] == "GSW"][0]

In [11]:
gsw_id = gsw['id']

In [13]:
warriors_gl = teamgamelog.TeamGameLog(gsw_id)

In [14]:
warriors_20_df = warriors_gl.get_data_frames()[0]

In [15]:
warriors_20_df[warriors_20_df['MATCHUP'].str.contains('SAS')]

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
10,1610612744,0022000378,"FEB 09, 2021",GSW @ SAS,W,13,12,0.520,240,42,91,0.462,17,40,0.425,13,19,0.684,10,40,50,26,8,4,10,19,114
11,1610612744,0022000371,"FEB 08, 2021",GSW @ SAS,L,12,12,0.500,240,38,86,0.442,13,37,0.351,11,15,0.733,6,43,49,29,4,6,20,19,100
21,1610612744,0022000223,"JAN 20, 2021",GSW vs. SAS,W,8,6,0.571,240,46,91,0.505,15,38,0.395,14,20,0.700,10,44,54,31,6,2,14,22,121


In [16]:
gamelog_steph = playergamelog.PlayerGameLog(steph_id)

In [17]:
gamelog_steph_df = gamelog_steph.get_data_frames()[0]

In [18]:
gamelog_steph_df.head(2)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,201939,0022000527,"FEB 28, 2021",GSW @ LAL,L,26,5,13,0.385,2,7,0.286,4,5,0.8,0,4,4,7,3,0,3,2,16,-24,1
1,22020,201939,0022000511,"FEB 26, 2021",GSW vs. CHA,W,36,8,15,0.533,3,8,0.375,10,10,1.0,0,4,4,9,1,0,4,1,29,23,1


In [19]:
gamelog_steph_df['PAR'] = gamelog_steph_df['PTS'] + gamelog_steph_df['AST'] + gamelog_steph_df['REB'] 

In [20]:
gamelog_steph_df.head(2)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PAR
0,22020,201939,0022000527,"FEB 28, 2021",GSW @ LAL,L,26,5,13,0.385,2,7,0.286,4,5,0.8,0,4,4,7,3,0,3,2,16,-24,1,27
1,22020,201939,0022000511,"FEB 26, 2021",GSW vs. CHA,W,36,8,15,0.533,3,8,0.375,10,10,1.0,0,4,4,9,1,0,4,1,29,23,1,42


In [21]:
gamelog_steph_df['PAR'].mean()

41.294117647058826

In [22]:
brogdon = [player for player in player_dict if player['full_name'] == 'Malcolm Brogdon'][0]

In [23]:
brogdon_id = brogdon['id']

In [24]:
gamelog_brogdon_20 = playergamelog.PlayerGameLog(brogdon_id, '2020')
gamelog_brogdon_19 = playergamelog.PlayerGameLog(brogdon_id, '2019')
gamelog_brogdon_18 = playergamelog.PlayerGameLog(brogdon_id, '2018')
gamelog_brogdon_17 = playergamelog.PlayerGameLog(brogdon_id, '2017')

In [25]:
gamelog_brogdon_20_df = gamelog_brogdon_20.get_data_frames()[0]
gamelog_brogdon_19_df = gamelog_brogdon_19.get_data_frames()[0]
gamelog_brogdon_18_df = gamelog_brogdon_18.get_data_frames()[0]
gamelog_brogdon_17_df = gamelog_brogdon_17.get_data_frames()[0]

In [26]:
brog_17to20_df = pd.concat([gamelog_brogdon_17_df, gamelog_brogdon_18_df, gamelog_brogdon_19_df, gamelog_brogdon_20_df])

In [27]:
brog_17to20_df['GAME_DATE'] = pd.to_datetime(brog_17to20_df['GAME_DATE'])

In [28]:
brog_17to20_df.sort_values(by='GAME_DATE', ascending=False,inplace=True)

In [29]:
brog_17to20_df.head(2)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,1627763,0022000530,2021-03-01,IND @ PHI,L,29,9,17,0.529,0,3,0.000,2,2,1.0,0,4,4,4,0,0,0,2,20,-16,1
1,22020,1627763,0022000505,2021-02-26,IND @ BOS,L,33,5,17,0.294,2,7,0.286,0,0,0.0,2,4,6,6,0,0,1,4,12,-4,1


In [30]:
brog_17to20_df['PAR'] = brog_17to20_df['PTS'] + brog_17to20_df['AST'] + brog_17to20_df['REB'] 

In [31]:
brog_17to20_df.head(2)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,PAR
0,22020,1627763,0022000530,2021-03-01,IND @ PHI,L,29,9,17,0.529,0,3,0.000,2,2,1.0,0,4,4,4,0,0,0,2,20,-16,1,28
1,22020,1627763,0022000505,2021-02-26,IND @ BOS,L,33,5,17,0.294,2,7,0.286,0,0,0.0,2,4,6,6,0,0,1,4,12,-4,1,24


In [32]:
brog_vs_bkn = brog_17to20_df[brog_17to20_df['MATCHUP'].str.contains('BKN')]

In [33]:
li_brog_vs_bkn = brog_vs_bkn['Game_ID'].to_list()

In [36]:
def oppFinder(row):
    return row.split()[-1]

In [37]:
oppFinder('IND vs. UTA')

'UTA'

In [38]:
brog_17to20_df['OPPONENT'] = brog_17to20_df['MATCHUP'].apply(oppFinder)

In [40]:
brog_17to20_df.groupby('OPPONENT').agg(
    AVG_PLUSMINUS = ('PLUS_MINUS', 'mean'),
    AVG_REB = ('REB', 'mean'),
    AVG_STL = ('STL', 'mean'),
    AVG_PTS = ('PTS', 'mean'),
    AVG_AST = ('AST', 'mean'),
    AVG_PAR = ('PAR', 'mean')
 ).round(2)

,AVG_PLUSMINUS,AVG_REB,AVG_STL,AVG_PTS,AVG_AST,AVG_PAR
OPPONENT,,,,,,
ATL,5.83,4.67,1.00,17.00,6.17,27.83
BKN,5.00,6.20,1.20,15.20,5.00,26.40
BOS,1.00,4.60,0.50,17.00,4.80,26.40
CHA,2.75,4.08,0.75,17.08,5.42,26.58
CHI,5.80,5.40,0.60,17.80,4.80,28.00
CLE,4.50,3.75,1.12,20.75,5.25,29.75
DAL,-3.33,3.83,1.50,15.83,3.33,23.00
DEN,5.00,5.67,0.67,18.33,5.33,29.33
DET,3.60,5.40,0.90,15.70,5.70,26.80


---

In [43]:
with open('json_nba.json') as f:
    data = json.load(f)

In [44]:
json_data = ast.literal_eval(data)

In [45]:
json = json_data[0]

In [48]:
events = json['events'][0]

In [50]:
df = pd.json_normalize(events, sep='_')

In [51]:
df

,id,description,type,link,status,sport,startTime,live,awayTeamFirst,denySameGame,teaserAllowed,competitionId,notes,numMarkets,lastModified,competitors,displayGroups
0,8301268,Golden State Warriors @ San Antonio Spurs,GAMEEVENT,/basketball/nba/golden-state-warriors-san-anto...,U,BASK,1612920900000,False,True,NO,True,2958468,,149,1612912812375,"[{'id': '8301268-204', 'name': 'San Antonio Sp...","[{'id': '100-97', 'description': 'Game Lines',..."


In [53]:
competitors = pd.json_normalize(df['competitors'][0])
#competitors of the current game matchup

In [54]:
competitors

,id,name,home
0,8301268-204,San Antonio Spurs,True
1,8301268-11757531,Golden State Warriors,False


In [55]:
displayGroups = pd.json_normalize(df['displayGroups'][0])
#types of betting (lines, game&player props, periods)

In [56]:
displayGroups

,id,description,defaultType,alternateType,markets,order
0,100-97,Game Lines,True,False,"[{'id': '144048853', 'descriptionKey': 'Head T...",1
1,100-101,Period/Alternate Lines,False,False,"[{'id': 'G-2W-HCAP.Handicap - Asian.209', 'des...",4
2,100-104,Game Props,False,False,"[{'id': '144048848', 'descriptionKey': 'First ...",6
3,100-105,Player Props,False,False,"[{'id': '144144463', 'descriptionKey': 'Will {...",7


In [58]:
player_props = pd.json_normalize(displayGroups[displayGroups['description'] == 'Player Props']['markets'][3])

In [60]:
player_props.drop(['status','singleOnly','notes', 'period.abbreviation','period.live'],inplace=True,axis=1)

In [61]:
player_props.head(2)

,id,descriptionKey,description,key,marketTypeId,outcomes,period.id,period.description,period.main
0,144144463,Will {Dejounte Murray (SAS)} record a Double-D...,Will Dejounte Murray (SAS) record a Double-Dou...,GAME-PROP-12,-1093,"[{'id': '773861571', 'description': 'Yes', 'st...",209,Game,True
1,144144458,Will {DeMar DeRozan (SAS)} record a Double-Dou...,Will DeMar DeRozan (SAS) record a Double-Double?,GAME-PROP-12,-1093,"[{'id': '773861561', 'description': 'Yes', 'st...",209,Game,True


In [64]:
pd.json_normalize(player_props['outcomes'][0])

,id,description,status,type,price.id,price.american,price.decimal,price.fractional,price.malay,price.indonesian,price.hongkong
0,773861571,Yes,O,X,7077383005,+235,3.350,47/20,-0.43,2.35,2.35
1,773861572,No,O,X,7077383006,-330,1.30303,10/33,0.30,-3.30,0.30


In [65]:
player_props.head(1)

,id,descriptionKey,description,key,marketTypeId,outcomes,period.id,period.description,period.main
0,144144463,Will {Dejounte Murray (SAS)} record a Double-D...,Will Dejounte Murray (SAS) record a Double-Dou...,GAME-PROP-12,-1093,"[{'id': '773861571', 'description': 'Yes', 'st...",209,Game,True


In [67]:
player_props['outcome_ids'] = player_props['outcomes'].apply(lambda x: "|".join(i['id'] for i in x))

In [68]:
player_props['player_prop_odds'] = player_props['outcomes'].apply(lambda x: "|".join(i['price']['american'] for i in x))

In [70]:
player_props['yes'] = player_props['player_prop_odds'].apply(lambda x: x.split('|')[0])
player_props['no'] = player_props['player_prop_odds'].apply(lambda x: x.split('|')[1])

In [72]:
player_props['outcome_yes_id'] = player_props['outcome_ids'].apply(lambda x: x.split('|')[0])
player_props['outcome_no_id'] = player_props['outcome_ids'].apply(lambda x: x.split('|')[1])

In [74]:
player_props.drop(['outcome_ids', 'player_prop_odds'],axis=1,inplace=True)

In [77]:
dejounte = [player for player in player_dict if player['full_name'] == 'Dejounte Murray'][0]
demar = [player for player in player_dict if player['full_name'] == 'DeMar DeRozan'][0]

In [78]:
player_props.iloc[25]['outcomes'][1]

{'id': '773861620',
 'description': 'Under',
 'status': 'O',
 'type': 'U',
 'competitorId': '8301268-11766057',
 'price': {'id': '7077806507',
  'handicap': '42.5',
  'american': '-110',
  'decimal': '1.909091',
  'fractional': '10/11',
  'malay': '0.91',
  'indonesian': '-1.10',
  'hongkong': '0.91'}}

In [82]:
def isHandicap(row):
    for i in row:
        try:
            return i['price']['handicap']
        except (KeyError):
            return np.nan

In [83]:
player_props['handicap'] = player_props['outcomes'].apply(isHandicap)

In [84]:
player_props['player_prop_odds'] = player_props['outcomes'].apply(lambda x: "|".join(i['price']['american'] for i in x))

In [86]:
player_props.head(1)

,id,descriptionKey,description,key,marketTypeId,outcomes,period.id,period.description,period.main,yes,no,outcome_yes_id,outcome_no_id,handicap,player_prop_odds
0,144144463,Will {Dejounte Murray (SAS)} record a Double-D...,Will Dejounte Murray (SAS) record a Double-Dou...,GAME-PROP-12,-1093,"[{'id': '773861571', 'description': 'Yes', 'st...",209,Game,True,+235,-330,773861571,773861572,NaN,+235|-330


In [87]:
player_props['player_name'] = player_props['descriptionKey'].apply(lambda s: s.split('{',1)[1].split('}')[0]).apply(lambda s: s.split('(')[0])

In [88]:
player_props['player_team'] = player_props['descriptionKey'].apply(lambda s: s.split('{',1)[1].split('}')[0]).apply(
lambda s: s.split('(',1)[1].split(')')[0])

In [90]:
unique_team_names = list(player_props['player_team'].unique())

In [91]:
unique_team_names

['SAS', 'GSW']

In [92]:
def getOpponent(row):
    for i in unique_team_names:
        if i == row:
            return unique_team_names[-1]
        else:
            return unique_team_names[0]

In [93]:
player_props['opp_team'] = player_props['player_team'].apply(getOpponent)

In [94]:
player_props.iloc[0]['descriptionKey']

'Will {Dejounte Murray (SAS)} record a Double-Double?'

In [95]:
player_props.iloc[-7]['descriptionKey'].split(' - {')

['Total Points, Rebounds and Assists', 'Stephen Curry (GSW)} (NBA)']

In [96]:
gsw_sas = teamgamelog.TeamGameLog(gsw_id)

In [97]:
gsw_sas = gsw_sas.get_data_frames()[0]

In [98]:
gsw_sas.head()

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612744,0022000527,"FEB 28, 2021",GSW @ LAL,L,19,16,0.543,240,35,86,0.407,8,34,0.235,13,19,0.684,3,32,35,26,10,4,20,25,91
1,1610612744,0022000511,"FEB 26, 2021",GSW vs. CHA,W,19,15,0.559,240,48,79,0.608,11,24,0.458,23,25,0.920,4,33,37,38,10,3,23,19,130
2,1610612744,0022000489,"FEB 24, 2021",GSW @ IND,W,18,15,0.545,240,39,83,0.470,5,26,0.192,28,31,0.903,8,33,41,27,10,4,18,25,111
3,1610612744,0022000483,"FEB 23, 2021",GSW @ NYK,W,17,15,0.531,240,40,89,0.449,13,38,0.342,21,24,0.875,7,34,41,30,9,8,10,18,114
4,1610612744,0022000459,"FEB 20, 2021",GSW @ CHA,L,16,15,0.516,240,37,85,0.435,13,39,0.333,13,17,0.765,9,30,39,23,10,3,14,12,100


In [101]:
game_datetime = pd.to_datetime(re.findall(r'\d+', df['link'][0])[0][:8])

In [102]:
game_datetime

Timestamp('2021-02-09 00:00:00')

In [103]:
player_props['GAME_DATE'] = game_datetime

In [104]:
player_props.head(1)

,id,descriptionKey,description,key,marketTypeId,outcomes,period.id,period.description,period.main,yes,no,outcome_yes_id,outcome_no_id,handicap,player_prop_odds,player_name,player_team,opp_team,GAME_DATE
0,144144463,Will {Dejounte Murray (SAS)} record a Double-D...,Will Dejounte Murray (SAS) record a Double-Dou...,GAME-PROP-12,-1093,"[{'id': '773861571', 'description': 'Yes', 'st...",209,Game,True,+235,-330,773861571,773861572,NaN,+235|-330,Dejounte Murray,SAS,GSW,2021-02-09


In [105]:
player_props['MATCHUP'] = player_props['player_team'] + ' @ ' + player_props['opp_team']

In [107]:
prop_action = ['Double-Double, Total Points', 'Total Rebounds and Assists',
              'Total Points, Rebounds and Assists', 'Total Made 3 Point Shots']

for i in player_props['descriptionKey']:
    if i in prop_action:
        print(prop_action)
    else:
        print('not here')

In [108]:
def getPlayerStats(player_name):
    gets_player_name = [player for player in player_dict if player['full_name'] == player_name][0]
    player_name_id = gets_player_name['id']
    df_player = playergamelog.PlayerGameLog(player_name_id).get_data_frames()[0]
    df_player['PAR'] = df_player['PTS'] + df_player['AST'] + df_player['REB']
    df_player['PA'] = df_player['PTS'] + df_player['AST']
    return df_player

---

In [110]:
players_df = pd.DataFrame(data.players)

In [111]:
players_df.columns = ['player_id','player_lastname','player_first_name','player_fullname','player_active']

In [112]:
active_players_df = players_df[players_df['player_active'] == True]

In [114]:
teams_df = pd.DataFrame(data.teams)

In [115]:
teams_df.columns = ['team_id','team_abb','team_nickname','team_yr','team_city','team_fullname','team_state']

In [116]:
teams_df.drop('team_yr',inplace=True,axis=1)

In [119]:
teams_df[teams_df['team_abb'] == 'DAL']['team_id']

5    1610612742
Name: team_id, dtype: int64

In [120]:
def get_teams_id(teams_abb):
    teams_list = []
    for team in teams_abb:
        teams_list.append(teams_df[teams_df['team_abb'] == team]['team_id'].item())
    return teams_list

def get_teams_abb(teams_id):
    teams_list = []
    for team in teams_id:
        teams_list.append(teams_df[teams_df['team_id'] == team]['team_abb'].item())
    return teams_list

In [121]:
get_teams_id(['DAL', 'NOP'])

[1610612742, 1610612740]

In [122]:
get_teams_abb([1610612742, 1610612740])

['DAL', 'NOP']

In [123]:
df1 = teams_df[teams_df['team_abb'] == 'BOS']
df2 = teams_df[teams_df['team_abb'] == 'NOP']

In [124]:
pd.concat([df1,df2])

,team_id,team_abb,team_nickname,team_city,team_fullname,team_state
1,1610612738,BOS,Celtics,Boston,Boston Celtics,Massachusetts
3,1610612740,NOP,Pelicans,New Orleans,New Orleans Pelicans,Louisiana


In [125]:
def getTeamRecentGames(team_abb):
    gets_team_name = [team for team in team_dict if team['abbreviation'] == team_abb][0]
    team_name_id = gets_team_name['id']
    df_team = teamgamelog.TeamGameLog(team_name_id).get_data_frames()[0]
    return df_team

In [126]:
getTeamRecentGames('NOP').head(2)

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612740,0022000533,"MAR 01, 2021",NOP vs. UTA,W,15,19,0.441,240,52,92,0.565,7,11,0.636,18,23,0.783,10,43,53,27,5,5,13,15,129
1,1610612740,0022000514,"FEB 27, 2021",NOP @ SAS,L,14,19,0.424,240,40,91,0.440,9,32,0.281,25,31,0.806,17,35,52,27,3,2,10,17,114


In [127]:
getTeamRecentGames('DAL').head(2)

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612742,0022000529,"MAR 01, 2021",DAL @ ORL,W,17,16,0.515,240,47,90,0.522,17,46,0.370,19,23,0.826,7,39,46,29,6,6,8,12,130
1,1610612742,0022000519,"FEB 27, 2021",DAL @ BKN,W,16,16,0.500,240,45,86,0.523,14,36,0.389,11,17,0.647,6,30,36,23,12,8,8,15,115
